# Data Generation


## Importing Libraries

In [47]:
import cv2
import imutils
import os

### Defining Global Variables

In [2]:
bg = None

## Running Average Function

In [3]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

## Segementaion Method

In [4]:
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(),
                                 cv2.RETR_EXTERNAL,
                                 cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

## Capturing the Data

In [30]:
# initialize weight for running average
aWeight = 0.5

# region of interest (ROI) coordinates
top, right, bottom, left = 10, 350, 225, 590

# initialize num of frames
num_frames = 0

## Gestures List

In [60]:
start_recording = False
gestures = ['fist', 'palm', 'swing']
data_dir = 'data/train'
current = 0
no_of_images = 100
start_image_num = 10

### Generating Directories

In [32]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
if not os.path.isdir(data_dir+ '/' + gestures[current]):
    os.mkdir(data_dir+ '/' + gestures[current])

In [51]:
end_image_num = start_image_num + no_of_images
image_num = start_image_num

# get the reference to the webcam
camera = cv2.VideoCapture(0)

In [52]:
# check if camera is open
if not camera.isOpened():
    print("Cannot open camera")
    print(camera)

In [56]:
# debugging
"""
cap = cv2.VideoCapture(0)
while True:

    ret, frame = cap.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
"""

"\ncap = cv2.VideoCapture(0)\nwhile True:\n\n    ret, frame = cap.read()\n    cv2.imshow('frame',frame)\n    if cv2.waitKey(1) & 0xFF == ord('q'):\n        break\n\ncap.release()\ncv2.destroyAllWindows()\n"

In [53]:
# keep looping, until interrupted
while True:
    # get the current frame
    grabbed, frame = camera.read()
    if grabbed == True:

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
            # print(num_frames, end =' frame.')
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(
                    clone, [segmented + (right, top)], -1, (0, 0, 255))
                if start_recording:

                    # Mention the directory in which you wanna store the images 
                    # followed by the image name
                    img_save = data_dir + "/" + gestures[current] + "/" + gestures[current] + "_"
                    cv2.imwrite(img_save + str(image_num) + '.png', thresholded)
                    image_num += 1
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q") or image_num > end_image_num:
            break

        if keypress == ord("s"):
            start_recording = True
    else:
        print("Error, Please check your camera")
        print(camera)
        break
camera.release()
cv2.destroyAllWindows()

## Resizing the Generated Images

In [57]:
# importing necessary libraries
from PIL import Image

In [59]:
def resizeImage(imageName):
    basewidth = 100
    img = Image.open(imageName)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    img.save(imageName)

for i in range(start_image_num, end_image_num + 1):
    # Mention the directory in which you want to
    # resize the images followed by the image name
    img_save = data_dir + "/" + gestures[current] + "/" + gestures[current] + "_"
    resizeImage(img_save + str(i) + '.png')